### This Jupyter Notebook demonstrates the key steps and corresponding Python code used for ABoVE Aboveground Biomass (AGB) mapping across the Arctic and Boreal regions of North America. For any questions, please contact wliangpudding@gmail.com or liangwan127@outlook.com

## Step 1: calculate annual seasonal spectral featurs from CCDC  

### Fitted CCDC parameters were obtained from GEE assets 'projects/CCDC/measures/v1_overlap''

### Import key python packages

In [1]:
import rasterio
import pandas as pd
import os
import geopandas as gpd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time

### Define functions needed to derive annual seasonal spectral features from CCDC 

In [2]:
MS_TO_DAYS = 86400000  # Number of milliseconds in a day
EPOCH_DAYS = 719163    # Days from Julian calendar start (0000-01-01) to Unix epoch (1970-01-01)

In [4]:
def precompute_indices(ccd_names):
    startBands_indices = [i for i, name in enumerate(ccd_names) if '_tStart' in name]
    endBands_indices = [i for i, name in enumerate(ccd_names) if '_tEnd' in name]
    return startBands_indices, endBands_indices

def filterCoefs(ccd_names, ccd_results, date, band, coef, startBands_indices, endBands_indices):

    coefBands_indices = [index for index, name in enumerate(ccd_names) if ((band in name)&(name.split('_')[-1]==coef))]
    start_bands = ccd_results[startBands_indices,:,:]
    end_bands = ccd_results[endBands_indices,:,:]
    coef_bands = ccd_results[coefBands_indices,:,:]

    start_bands[start_bands ==0.] = np.nan
    end_bands[end_bands ==0.] = np.nan

    segment_match_after = end_bands > date
    first_true_indices = np.argmax(segment_match_after, axis=0)
    bool_mask = segment_match_after == True
    no_true_value = ~np.any(bool_mask, axis=0)

    rows, cols = np.indices((coef_bands.shape[1], coef_bands.shape[2]))
    # Use these indices to select from coef_bands
    out_coef_after = coef_bands[first_true_indices, rows, cols]
    out_coef_after = np.where(no_true_value, np.nan, out_coef_after)

    segment_match_before = start_bands < date
    first_false_indices = np.argmin(segment_match_before, axis=0)
    last_true_indices = first_false_indices - 1
    rows, cols =  np.indices((coef_bands.shape[1], coef_bands.shape[2]))
    out_coef_before = coef_bands[last_true_indices, rows, cols]

    nan_mask = np.isnan(out_coef_after)
    out_coef_after[nan_mask] = out_coef_before[nan_mask]
    return out_coef_after

def getCoef(ccd_names, ccd_results, date, band_list, coef, startBands_indices, endBands_indices):
  coefs= [filterCoefs(ccd_names, ccd_results, date, band, coef, startBands_indices, endBands_indices) for band in band_list]
  return np.stack(coefs, axis=-1) if len(band_list) > 1 else coefs[0]

def getMultiCoefs(ccd_names, ccd_results, date, band_list, coef_list, cond, seg_names, behavior, startBands_indices, endBands_indices):
    # This function will use the previously converted getCoef and filterCoefs functions
    out_coefs = [getCoef(ccd_names, ccd_results, date, band_list, coef,  startBands_indices, endBands_indices) for coef in coef_list]
    out_coefs = np.stack(out_coefs, axis=-1)
    return out_coefs


def getSyntheticForYear(ccd_names, image, date,  bands, segs, startBands_indices, endBands_indices):
  print('******************** getting data for year {} ******************'.format(date))
  # Constants
  PI2 = 2.0 * np.pi
  omega = PI2

  # Convert date to a numeric format (e.g., days since some reference date)
  tfit = date
  # Create synthetic image (array in this case)
  image_t = np.array([[1], [tfit], [np.cos(tfit * omega)], [np.sin(tfit * omega)],
                      [np.cos(tfit * omega * 2)], [np.sin(tfit * omega * 2)],
                      [np.cos(tfit * omega * 3)], [np.sin(tfit * omega * 3)]]).astype(float)
  image_t = image_t.T
  COEFS = ["INTP", "SLP", "COS", "SIN", "COS2", "SIN2", "COS3", "SIN3"]
  segnames = ['S'+ str(t) for t in segs]
  new_params = getMultiCoefs(ccd_names, image, date, bands, COEFS, False, segs, 'auto', startBands_indices, endBands_indices)

  synthetic_result_bands= [np.tensordot(new_params[:,:,i,:], image_t, axes=([2],[1])) for i in range(new_params.shape[2])]
  synthetic_result_bands = np.dstack(synthetic_result_bands)
  return (synthetic_result_bands*10000).astype(np.int16)

def getSyntheticMultiYears(ccd_names, image, dates, bandList, segs, startBands_indices, endBands_indices):
  multi_bands_years = [getSyntheticForYear(ccd_names, image, date,  bandList, segs, startBands_indices, endBands_indices) for date in dates]
  multi_bands_years= np.dstack(multi_bands_years)
  multi_bands_years= np.moveaxis(multi_bands_years, -1, 0)
  return multi_bands_years

"""#### Define parallel function"""
def ccdc2BandsVIsByYear(file_path):
  start_time = time.time()

  ccdImage = rasterio.open(file_path)
  meta = ccdImage.meta
  meta.update({
      'dtype':'int16'
      })
  ccdImage = ccdImage.read()
  #ccdImage = ccdImage/10000
  print(ccdImage.shape)
  startBands_indices, endBands_indices = precompute_indices(ccd_names)
  syn_bands = getSyntheticMultiYears(ccd_names, ccdImage, dates,  bands, segs, startBands_indices, endBands_indices)
  #syn_bands = syn_bands*10000
  if calculateVIs:
    allBands = bands + VIs
    band_names= ["Year_" + (str(year))[:4] +'_' + band for year in dates for band in allBands]
  else:
    band_names= ["Year_" + (str(year))[:4] +'_' + band for year in dates for band in bands]

  years = [str(year)[:4] for year in dates]
  for year in years:
    bands_indices = [i for i, name in enumerate(band_names) if str(year) in name]
    year_syn_bands = syn_bands[bands_indices,...]
    meta.update(count= year_syn_bands.shape[0])
    out_name = 'Summer_Year_'+ str(year) + '_'+ 'Bands_' + (file_path.split('/'))[-1].split('_')[-1]
    year_band_names = [band_names[i] for i in bands_indices]
    year_band_names = [nm[10:] for nm in year_band_names]
    output_path = os.path.join(out_path, out_name)
    with rasterio.open(output_path, 'w', **meta) as dst:
      dst.write(year_syn_bands)
      dst.descriptions= year_band_names
  print('time to complete: {} minutes'.format((time.time() - start_time)/60))

#### Among all the functions above, ccdc2BandsVIsByYear is the wrapper function and the input is supposed to be a geotiff with fitted CCDC coefficents. Function getSyntheticMultiYears is a key function that takes all relevant parameters as input and then return spectral features for a given date. The output is synthethic reflectance value in a integer format with scalar factor of 10000. A detailed description on all input parameters for function getSyntheticMultiYears is described below. 
1. ccd_names: a list of string to indicates names of different bands of downloaded CCDC parametrs;
2. image: 3D numpy array for the CCDC parameters, 1st dimension is for band;
3. dates: float, date to derive spectral feature, e.g., 2024.5 means mid of year 2024;
4. bandList: list of string to indicate names of bands to derive spectral features, e.g. ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"];
5. segs: list of string to indicate names of each segment, e.g. ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14", "S15"]
; 
6. startBands_indice and 7.  endBands_indice: derived by ccdc2BandsVIsByYear function based on band names of downloaded CCDC parameters.s

### Example to apply the wrapper function to calculate spectral features 

In [7]:
ccd_names =  pd.read_csv('CCDC_names_new.csv')
ccd_names = list(ccd_names['CCDC_names'])
bands = ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"]
segs = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14", "S15"]
out_path = '/ts_output/'

# Here an example to calculate mid of year spectral features for 2019 and 2020
years = [2019, 2020]
dates = []
for year in years:
    if year % 4 == 0:
        dates.append(year + 213/366)
    else:
        dates.append(year + 212/365)

print(dates)

[2019.5808219178082, 2020.5819672131147]


In [ ]:
ccdc2BandsVIsByYear('CCDC_ts.tif')

## Step 2: Variable selection and parameter tunning

### Define functions 

In [9]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import explained_variance_score
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split

# variable selection function using xgbm and recursive feature selection
def xgbm_var_sel(dt_x, dt_y, n_top=35):
    rf =  XGBRegressor(max_depth=14, min_child_weight=2, n_estimators=200, n_jobs=20, learning_rate=0.03, method='approx', max_bin = 250)
    selector = RFE(estimator=rf, n_features_to_select=n_top, step=2)
    selector = selector.fit(dt_x, dt_y)
    
    # Get the ranking of features
    ranking = selector.ranking_
    
    # Create a DataFrame for better visualization
    feature_ranking = pd.DataFrame({'Feature': dt_x.columns, 'Ranking': ranking})
    feature_ranking = feature_ranking.sort_values(by='Ranking')
    
    return list(feature_ranking['Feature'])[:n_top]
    
def train_evaluate(x_tr, x_val, y_tr, y_val, n_tree, max_depth, lr, sub_sp, tree_method, max_bin, colsample):
    
  fit_md=   XGBRegressor( n_estimators=n_tree, max_depth=max_depth, learning_rate= lr,  colsample_bytree= colsample,subsample= sub_sp, tree_method=tree_method, max_bin=max_bin).fit(x_tr, y_tr)
  pred_val = fit_md.predict(x_val)
  f1_val= explained_variance_score(y_val, pred_val)
  #f1_val= -np.mean(abs(fit_md.predict(x_val) - np.array(y_val['Lidar_AGB'])))
  return f1_val

def objective(trial=20):
    params = {
             'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
              'n_tree': trial.suggest_int("n_tree", 100, 400, step=100),
              'max_depth': trial.suggest_int("max_depth", 4, 14, step=2),
              'subsample': trial.suggest_float("subsample", 0.4, 1.0, step=0.2),
              'tree_method': trial.suggest_categorical('tree_method', ['exact', 'approx']),
              'max_bin': trial.suggest_int('max_bin', 200, 250, step=50),
              'colsample_bytree': trial.suggest_float("colsample_bytree", 0.4, 1, step=0.2)
              }
    n_tree = params['n_tree']
    max_depth= params['max_depth']
    sub_sp= params['subsample']
    lr = params['learning_rate']
    tree_method= params['tree_method']
    max_bin= params['max_bin']
    colsample = params['colsample_bytree']
    accuracy= train_evaluate(x_tr, x_val, y_tr, y_val, n_tree, max_depth, lr, sub_sp, tree_method, max_bin, colsample)
    return accuracy

### Example to select 35 variables then fine tune parameters

In [ ]:
select_feas = xgbm_var_sel(dt_x, dt_y, n_top=35) # dt_x and dt_y are predictors and reponse from training data 

In [ ]:
x_tr, x_val, y_tr, y_val = train_test_split(dt_x[select_feas], dt_y, test_size=0.2)
para_opt = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
para_opt.optimize(objective, n_trials=100, n_jobs=10)
output = para_opt.trials_dataframe()
output = output.sort_values(['value'], ascending=False).reset_index(drop=True)
nm_out = 'Paramter_tunning_XGBM.csv'
output.to_csv(nm_out , index=False)

## Step 3: Model training & saving 

In [ ]:
def uncert_train_model(dt_train, i, xgbm_parameters_path = 'Paramter_tunning_XGBM.csv',y_variable_nm = 'AGB', selected_features= ['RED_summer','BLUE_summer','NDVI_summer'], model_out_path = ''):
    """
    This code does the following tasks:
    1) Simulates 3 types of uncertaties: sampling uncertainty using bootstrapping, allometry uncertainty, and CCDC/COLD uncertainty 
    2) train a XGBM regression with uncertainty-added data, 
    3) and save the model
    Args:
        dt_train: DataFrame containing the balanced training data,
        i: integer to indicate which interation this run is, 0 means first itervation, n will be used in the name of the output model,
        y_variable_nm: string to indicate column name for y variable,
        selected_features: list of string to indicate name of features for model training,
        model_out_path: string to indciate path where you want to save the trained model,
        out_name: string to indicate name of the output model.

    Returns:
       no return, but save the trained model in given location
    """
    
    ## load training data using bootstrapping mehtod, bootsrapping_train is a self-defined function 
    random_state = n

    ## Add allometry uncertainty in y variable using Monte Carlo to simulate normal-distributed random error
    np.random.seed(random_state)
    # column y_variable_nm+'_std' is the uncertainty, which is used as standard deviation of the normally distributed random error 
    dt_train['y_error'] = np.random.normal(loc=0, scale= dt_train[y_variable_nm+'_std'] , size = dt_train.shape[0])
    dt_train['y2'] = dt_train['y_error'] + dt_train[y_variable_nm]

    dt_x = dt_train[selected_features]
    dt_y = dt_train[['y2']]

    paras = pd.read_csv('Paramter_tunning_XGBM.csv')
    model = XGBRegressor(n_estimators=paras['params_n_tree'].iloc[0], max_depth=paras['params_max_depth'].iloc[0], learning_rate=paras['params_learning_rate'].iloc[0], colsample_bytree=paras['params_colsample_bytree'].iloc[0], subsample=paras['params_subsample'].iloc[0], tree_method=paras['params_tree_method'].iloc[0], max_bin=paras['params_max_bin'].iloc[0],   n_jobs=10).fit(dt_x, dt_y)
    out_name =  f'XGBM_model_out{i}.json'
    model_path = model_out_path + '/' + out_name
    model.save_model(model_path)
    print(model_path + ' saved!')

In [ ]:
dta_sbb = bal_data_eco(eco)
out_name2 = eco + '_Nccdc_uncert_'+ str(n) + 'F30b.json'
uncert_train_model(dt_train, 0)

## Step 4: Model projection 

#### Load key pacakges and define functions 

In [ ]:
import os
import numpy as np, pandas as pd, geopandas as gpd
import rasterio
import time
import gzip
import shutil
from xgboost import  XGBRegressor
import xgboost as xgb
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import concurrent.futures
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import random


def calculate_VIs_raster(image):
    """
    This function computes vegetation indices.
    Args:
        image: numpy 3d array, with first dimension represents band. the code assumes 6 bands following below sequences: ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"] 

    Returns:
        numpy 3d array, with original input concated with computed vegetation indices
    """
    eps = 0.00001
    ##### NDVI #####
    ndvi = (image[3,:,:] - image[2,:,:])/(image[3,:,:] + image[2,:,:]+ eps)
    nbr = (image[3,:,:] - image[5,:,:])/(image[3,:,:] + image[5,:,:]+ eps)
    # gndvi = (image[3,:,:] - image[1,:,:])/(image[3,:,:] + image[1,:,:]+ eps)
    # ndwi = (image[1,:,:] - image[3,:,:])/(image[1,:,:] + image[3,:,:]+ eps)
    msi = image[4,:,:]/(image[3,:,:]+ eps)
    sr = image[3,:,:]/(image[2,:,:]+ eps)
    # evi = 2.5*(image[3,:,:] - image[2,:,:]) / (image[3,:,:] + (6 * image[2,:,:]) - (7.5 * image[0,:,:]) + 1)
    nbr2 = (image[4,:,:] - image[5,:,:])/(image[4,:,:] + image[5,:,:]+ eps)
    #ndmi = (image[3,:,:] - image[4,:,:])/(image[3,:,:] + image[4,:,:])
    ###### Tasseled Cap #####
    coefficients = {
          'brightness': [0.2043, 0.4158, 0.5524, 0.5741, 0.3124, 0.2303],
          'greenness': [-0.1603, -0.2819, -0.4934, 0.7940, -0.0002, -0.1446],
          'wetness': [0.0315, 0.2021, 0.3102, 0.1594, -0.6806, -0.6109]
       }
    # Applying the coefficients to the bands and summing them
    brightness = np.sum([image[i,:,:] * coefficients['brightness'][i] for i in range(6)], axis=0)
    greenness = np.sum([image[i,:,:] * coefficients['greenness'][i] for i in range(6)], axis=0)
    wetness = np.sum([image[i,:,:] * coefficients['wetness'][i] for i in range(6)], axis=0)
    image = np.concatenate((image,  ndvi.reshape(1, ndvi.shape[0], ndvi.shape[1]), 
                      msi.reshape(1, ndvi.shape[0], ndvi.shape[1]),  sr.reshape(1, ndvi.shape[0], ndvi.shape[1]), 
                      nbr2.reshape(1, ndvi.shape[0], ndvi.shape[1]),  nbr.reshape(1, ndvi.shape[0], ndvi.shape[1]),
                      brightness.reshape(1, ndvi.shape[0], ndvi.shape[1]), greenness.reshape(1, ndvi.shape[0], ndvi.shape[1])), axis=0)
    return image
    

def load_and_process_raster(file_path, name_suffix, anci_feas, spring_feas, early_summer_feas, mid_summer_feas, late_summer_feas, fall_feas):
    """
    this function load predictors from geotiffs, and return selected features as a 3d numpy array, and a list of string including names of predictors.
    
    Args:
        file_path: string, path of the predictor;
        name_suffix: string, one of the below options ['_ancillary', '_spring', '_early_summer', '_late_summer', 'fall'], representing different predictors, i.e. ancillary or spectral features, and which season;
        anci_feas: list, names of selected ancillary features;
        spring_feas: list, names of selected spectral features from spring season;
        early_summer_feas: list, names of selected spectral features from early summer season;
        mid_summer_feas: list, names of selected spectral features from mid summer season;
        late_summer_feas: list, names of selected spectral features from late summer season;
        fall_feas: list, names of selected spectral features from fall season;
    Returns:
        a 3d numpy array with loaded features, and the features names in the array.
    """
    with rasterio.open(file_path) as src:
        print(name_suffix)
        if not name_suffix == '_ancillary':
            data = src.read() / 10000
            data = calculate_VIs_raster(data)
            feature_names = list(src.descriptions)
            feature_names_n =[ 'NDVI', 'MSI', 'SR', 'NBR2', 'NBR', 'brightness', 'greenness']
            feature_names = feature_names + feature_names_n
            if not name_suffix == '_mid_summer':
                feature_names = [nm + name_suffix for nm in feature_names]
                
            print(feature_names)
            if name_suffix == '_spring':
                keep_indexes = [feature_names.index(x) for x in spring_feas]
                feas_nms = [feature_names[x] for x in keep_indexes]
            elif name_suffix == '_early_summer':
                keep_indexes = [feature_names.index(x) for x in early_summer_feas]      
                feas_nms = [feature_names[x] for x in keep_indexes]
            elif name_suffix == '_late_summer':
                keep_indexes = [feature_names.index(x) for x in late_summer_feas]    
                feas_nms = [feature_names[x] for x in keep_indexes]
            elif name_suffix == '_fall':
                keep_indexes = [feature_names.index(x) for x in fall_feas]  
                feas_nms = [feature_names[x] for x in keep_indexes]
            else:
                keep_indexes = [feature_names.index(x) for x in mid_summer_feas]       
                feas_nms = [feature_names[x] for x in keep_indexes]
        else:
            data = src.read() / 10000
            anci_nms = list(src.descriptions) 
            keep_indexes = [anci_nms.index(x) for x in anci_feas]
            feas_nms = [anci_nms[x] for x in keep_indexes]
            
        selected_data = data[keep_indexes,:,:]
        del data
        return selected_data, feas_nms
    
# Function to load model and make predictions
def load_and_predict(eco_model_index, data_2d):
    """
    This code load the saved model and make predictions on features.

    Args:
        eco_model_index: string, formatted as EcoregionName_ModelIndex, where EcoregionName is a string to indicate a particular ecoregion and ModelIndex is a integer to index unique id of the model for the ecoregion;
        data_2d: numpy 2d array with shape number_of_pixel*number_of_features
    
    Returns:
        return predicted AGB value * 100.
    """
    
    eco = eco_model_index.split('__')[0]
    model_index = eco_model_index.split('__')[1]
    model_path = f'/model_path/trained_MLs/{eco}_{model_index}.json'
    model = xgb.XGBRegressor()
    model.load_model(model_path)
    return model.predict(data_2d)*100
        
def pred_landsat(tile_year_nrep_eco_outpath):
    """
    This function is a wrapper function that predicts mean AGB and derive uncertainty for each tile, and then write the output as Cloud Optimized Geotiff.

    Args:
    tile_year_nrep_model: string, formatted as TileName_YearOfFeatures_NumberOfModels_EcoregionName_OutputPath.


    Returns:
    write AGB prediction and uncertainty as Cloud Optimized Geotiff.

    """
    tile = tile_year_nrep_model.split('_')[0]
    year = tile_year_nrep_model.split('_')[1]
    n_reps = int(tile_year_nrep_model.split('_')[2])
    eco = tile_year_nrep_model.split('_')[3]
    out_path = tile_year_nrep_model.split('_')[4]
    out_name = f'{eco}_AGB_Pred_Year_{year}_{tile}_{model}.tif'
   
    output_path = os.path.join(out_path, out_name)

    def load_feas(eco, model):
        model_path = f'model_path/trained_MLs/{eco}_0.json'
        model = xgb.XGBRegressor()
        model.load_model(model_path)
        return list(model.get_booster().feature_names)
        
    top_feature_names = load_feas(eco, model)
    anciA = ['TEMPERATURE','RAINFALL', 'DEM_GLO30', 'slope_GLO30', 'aspect_GLO30']
    anci_feas = [nm for nm in top_feature_names if nm in anciA]
    spring_feas = [nm for nm in top_feature_names if '_spring' in nm]
    early_summer_feas = [nm for nm in top_feature_names if '_early_summer' in nm]
    late_summer_feas = [nm for nm in top_feature_names if '_late_summer' in nm]
    fall_feas = [nm for nm in top_feature_names if '_fall' in nm]
    mid_summer_feas = list(set(top_feature_names) - set(anci_feas + spring_feas + early_summer_feas + late_summer_feas+fall_feas))
    
    mid_summer = f'spectral_feature_path/mid_summer_bands/Summer_Year_{year}_Bands_{tile}_Merged.tif'
    seasons = {
        "ancillary": f'ancillary_feature_path/Ancillary_{tile}_Merged.tif',
        "spring": f'spectral_feature_path/spring_bands/Spring_Year_{year}_Bands_{tile}_Merged.tif',
        "early_summer": f'spectral_feature_path/early_summer_bands/Early_Summer_Year_{year}_Bands_{tile}_Merged.tif',
        "mid_summer": f'spectral_feature_path/mid_summer_bands/Summer_Year_{year}_Bands_{tile}_Merged.tif',
        "late_summer": f'spectral_feature_path/late_summer_bands/Late_Summer_Year_{year}_Bands_{tile}_Merged.tif',
        "fall": f'spectral_feature_path/fall_bands/Fall_Year_{year}_Bands_{tile}_Merged.tif'
    }
    
    lc = f'land_cover_path/Year_{year}_{tile}_Merged.tif'
    

    if os.path.exists(output_path)& skip:
        print('skipping')
    else:
        print("Reading and processing ancillary and seasonal rasters")        
        with ThreadPoolExecutor(6) as executor:
            future_results = {season: executor.submit(load_and_process_raster,  file_path,  f"_{season}", anci_feas, spring_feas , early_summer_feas, mid_summer_feas, late_summer_feas, fall_feas)
                              for season, file_path in seasons.items()}
            seasonal_data = {season: future.result() for season, future in future_results.items()}

        seasonal_arrays = [data for data, _ in seasonal_data.values()]    
        seasonal_names = [name for _, name in seasonal_data.values()]
        seasonal_names = [name for sublist in seasonal_names for name in sublist]

        indices = [seasonal_names.index(item) for item in top_feature_names]

        geo_array = np.concatenate(seasonal_arrays, axis=0)
        del seasonal_arrays
        geo_array = geo_array[indices,:,:]
                    
        #print(len(top_feature_names))
        # Reshape for model input
        height, width = geo_array.shape[1], geo_array.shape[2]
        data_2d = geo_array.transpose(1, 2, 0).reshape(-1, len(top_feature_names))
        del geo_array
        # Predict using pre-trained models
        all_predictions = np.zeros((n_reps, data_2d.shape[0]), dtype=np.int32)
        
        
        # Use ThreadPoolExecutor to parallelize the prediction for each model
        model_indices = [eco + '__' + str(i) + model for i in range(n_reps)]
        with ThreadPoolExecutor(max_workers= 10) as executor:
            futures = {executor.submit(load_and_predict, j, data_2d): j for j in model_indices}
            for future in as_completed(futures):
                j = futures[future]
                j = int(j.split('__')[1][:-3])
                try:
                    all_predictions[j, :] = future.result()
                except Exception as exc:
                    print(f"Model {j} generated an exception: {exc}")
        
        # Calculate outputs and uncertainties
        predicted_rasters = np.mean(all_predictions, axis=0).reshape(height, width)
        predicted_std = np.std(all_predictions, axis=0).reshape(height, width)
        #percent_uncertainty = (predicted_std / np.maximum(predicted_rasters, 1e-6)) * 100
        out_all = np.stack((predicted_rasters, predicted_std), axis=0)
        #out_all = out_all*100

        print('post processing')
        if os.path.exists(lc)&(eco!='TUNDRA'):
            lc_src = rasterio.open(lc)
            lc_ra = lc_src.read(1)
            nodata_value_raster = lc_src.nodata
            nodata_locations = np.where(lc_ra == nodata_value_raster, True, False)
            out_all[:,(lc_ra == 1) | (lc_ra == 7)| (lc_ra == 9)] = 0
            out_all[:,lc_ra==0] = -999
        out_all[(out_all < 0) & (out_all > -990)] = 0

        # Write output raster
        print("Writing output raster")
        meta = rasterio.open(mid_summer).meta
        meta.update(count=out_all.shape[0], nodata=-999)
        meta.update({
          "driver": "COG",
          "dtype": "int32",
          'compress': 'lzw',
          'blockxsize': 512,
          'blockysize': 512,
          "tiled": True,
          "interleave": "band"})
        output_path = os.path.join(out_path, out_name)
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(out_all)
            dst.descriptions = ['pred_AGB', 'pred_std']
        print("Done!")


#### Example code to derive AGB and uncertainty predictions 

In [ ]:
pred_landsat('Bh008v015_2020_50_NWFM_/Landsat_prediction/output') # NWFM: Northwestern Forested Mountains

## Step 5: Post processing 

## Step 6: Quality flag layer